In [ ]:
import re
import os
import pandas as pd
import numpy as np

# combination of csvs with metadata in filename

the following patterns may be used:

* \_cell(.*?)_ -> celltype
* \_temp(.*?)_ -> temperature
* \_time(.*?)_ -> hybridization time
* \_sted(.*?)_ -> sted laser power
* \_shipment(.*?)_ -> shipment name
* \_slide(.*?)_ -> slide id : use (shipment nr)-(slide nr)
* \_target(.*?)_ -> targeted loci

In [ ]:
dr = '/Users/david/Desktop/VisitFebMar2017/'
csvs = [f for f in next(os.walk(dr))[2] if f.endswith('.csv') and not 'COMBINED' in f]


recell = re.compile('.*?_cell(.*?)_.*?')
retemp = re.compile('.*?_temp(.*?)_.*?')
retime = re.compile('.*?_time(.*?)_.*?')
rested = re.compile('.*?_sted(.*?)_.*?')
reshipment = re.compile('.*?_shipment(.*?)_.*?')
reslide = re.compile('.*?_slide(.*?)_.*?')
retarget = re.compile('.*?_target(.*?)_.*?')


res = pd.DataFrame()


for csv in csvs:
    
    tmp = pd.read_csv(os.path.join(dr, csv))
    
    cell = np.nan if recell.match(csv) == None else recell.match(csv).groups()[0]
    temp = np.nan if retemp.match(csv) == None else retemp.match(csv).groups()[0]
    time = np.nan if retime.match(csv) == None else retime.match(csv).groups()[0]
    sted = np.nan if rested.match(csv) == None else rested.match(csv).groups()[0]
    shipment = np.nan if reshipment.match(csv) == None else reshipment.match(csv).groups()[0]
    slide = np.nan if reslide.match(csv) == None else reslide.match(csv).groups()[0]
    target = np.nan if retarget.match(csv) == None else retarget.match(csv).groups()[0]
    
    tmp['cell'] = cell
    tmp['temp'] = temp
    tmp['time'] = time
    tmp['sted'] = sted
    tmp['shipment'] = shipment
    tmp['slide'] = slide
    tmp['target'] = target
        
    res = res.append(tmp, ignore_index=True)
    
res['len'] = np.sqrt((res.d01 - res.d02)**2 + (res.d11 - res.d12)**2 + (res.d21 - res.d22)**2)
res['len2d'] = np.sqrt((res.d01 - res.d02)**2 + (res.d11 - res.d12)**2 )
res.to_csv(os.path.join(dr, 'result_COMBINED.csv'), index=False)
    

# old code

In [ ]:
### for single samples

dr = '/Users/david/Desktop/AA_4th_shipment/'
csvs = [f for f in next(os.walk(dr))[2] if f.endswith('.csv')]
print(csvs)

p = re.compile('(.*?)_(.*?C)_(.*?s)\.csv')

with open(os.path.join(dr, 'out.csv'), 'w') as outfd:
    
    with open(os.path.join(dr, csvs[0]), 'r') as fstfd:
        outfd.write(fstfd.readline().strip() + ',cell,temp,time\n')
    
    for c in csvs:
        gr = list(re.match(p, c).groups())
        with open(os.path.join(dr, c), 'r') as fd:
            fd.readline()
            for line in fd.readlines():
                outfd.write(line.strip() + ',' + ','.join(gr) + '\n')

In [ ]:
### for multiple samples of same condition

dr = '/Users/david/Desktop/AA_combined_results/'
csvs = [f for f in next(os.walk(dr))[2] if f.endswith('.csv')]
print(csvs)

p = re.compile('(.*?)_(.*?C)_(.*?s)_([0-9]+)\.csv')

with open(os.path.join(dr, 'out.csv'), 'w') as outfd:
    
    with open(os.path.join(dr, csvs[0]), 'r') as fstfd:
        outfd.write(fstfd.readline().strip() + ',cell,temp,time,sample\n')
    
    for c in csvs:
        if re.match(p, c):
            gr = list(re.match(p, c).groups())
            with open(os.path.join(dr, c), 'r') as fd:
                fd.readline()
                for line in fd.readlines():
                    outfd.write(line.strip() + ',' + ','.join(gr) + '\n')

In [ ]:
# out.csv in subfolders
dr = '/Volumes/project_data/cooperation_data/Preliminary_projects/Darmstadt - gammaH2AX/20160723_SP5'

toprocess = list()
for d, ds, f in os.walk(dr):
    if 'res.csv' in f:
        toprocess.append(os.path.join(d, 'res.csv'))
        
with open(os.path.join(dr, 'combined.csv'), 'w') as resfd:
    
    with open(toprocess[0], 'r') as fstfd:        
        resfd.write(fstfd.readline().strip() + ',cond\n' )
        for line in fstfd.readlines():
            resfd.write(line.strip() + ',' + toprocess[0].split(os.path.sep)[-2] + '\n')
            
    for i in range(1, len(toprocess)):
        with open(toprocess[i], 'r') as infd:        
            infd.readline()
            for line in infd.readlines():
                resfd.write(line.strip() + ',' + toprocess[i].split(os.path.sep)[-2] + '\n')